In [1]:
import pandas as pd 
credentials = pd.read_csv('credentials.csv')
consumer_key = credentials['consumer_key'][0]
consumer_secret = credentials['consumer_secret'][0]
access_token = credentials['access_token'][0]
access_secret = credentials['access_secret'][0]
 
import tweepy
from tweepy import OAuthHandler
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True)

import warnings
warnings.simplefilter('ignore')

In [2]:
from pymongo import MongoClient
client = MongoClient()
news_tweets_extra = client.tweets_db.new_tweets_extra
news_tweets = client.tweets_db.new_tweets


In [4]:
news_tweets_extra.drop()

In [5]:
news_tweets_extra.count()

0

In [ ]:
#users = api.friends('@amnasri2', count=100)


In [6]:
import time
def collect_tweets(users):
    curr_count = news_tweets_extra.count()
    now = time.time()
    print("Started collecting tweets")
    for user in users:
        #tweets = api.user_timeline(screen_name = user.screen_name, count=200)
        try:
            tweets = tweepy.Cursor(api.user_timeline,screen_name=user.screen_name).items()
        except:
            continue
            
        for tw in tweets:

            if news_tweets.find_one({'id':tw.id}):
                break

            try:
                news_tweets_extra.insert_one(tw._json)
            except DuplicateKeyError:
                break
                
    new_count = news_tweets_extra.count()
    new_time = time.time() - now
    print("Collected %d tweets in %.2f seconds" % (new_count - curr_count, new_time))
    

In [7]:
import threading
import time


class MyThread(threading.Thread):
    def __init__(self, interval, task, args=[], kwargs={}):
        super(MyThread, self).__init__()
        self.stop_ = False
        self.interval = interval
        self.args = args
        self.kwargs = kwargs
        self.task = task
        self.start()
        
    def run(self):
        while not self.stop_:
            self.task(*self.args, **self.kwargs)
            time.sleep(self.interval)
            
    def stop(self):
        self.stop_ = True
        
users = api.friends('@amnasri2', count=100)
mythread = MyThread(15 * 60, collect_tweets, [users])

Started collecting tweets


In [164]:
news_tweets_extra.count()

53719

In [137]:
news_tweets_extra.count()

7682

Collected 2026 tweets in 324.68 seconds
Started collecting tweets
Collected 2035 tweets in 315.57 seconds


In [38]:
mythread.stop()